In [14]:
from pandas import read_csv
from datetime import datetime
import matplotlib as mpl
from matplotlib import pylab as plt
import matplotlib.dates as mdates
import pandas as pd
import collections
import seaborn as sns
import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pickle

In [15]:
# load data
def builData(cur):
    
    def parse(x):
        return datetime.strptime(x, '%Y.%m.%d')
    

    df = read_csv(cur + '.csv',sep='\t')

    df['<DATETIME>'] = df['<DATE>'] + ' ' + df['<TIME>']
#     df.index.name = '<DATETIME>'

    df.drop('<DATE>', axis=1, inplace=True)
    df.drop('<TIME>', axis=1, inplace=True)
    df.drop('<VOL>', axis=1, inplace=True)
    df.drop('<SPREAD>', axis=1, inplace=True)
    df.drop('<DATETIME>', axis=1, inplace=True)

    # mark all NA values with 0
#     curr_key = ['Canadian Dollar', 'Sterling', 'INR', 'US Dollar']

#     for curr in curr_key: 
#         df[curr].fillna(0, inplace=True)
#         df[curr] = df[curr].replace('-', 0) 
#     for j in range(5):
#         for curr in curr_key:
#             i = 0
#             for v in df[curr]:
#                 df[curr][i] = float(df[curr][i])
#                 if i < len(df[curr])-1 and df[curr][i] == 0:
#                     df[curr][i] = float(df[curr][i+1])        
#                 i += 1


    df.info()
    print(df.head(5))
    return df

In [16]:
cur_list = ['USDJPY', 'GBPUSD', 'EURUSD', 'USDCHF']
curr = cur_list[0]
df = builData(curr)
df.to_pickle(curr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 5 columns):
<OPEN>       2410 non-null float64
<HIGH>       2410 non-null float64
<LOW>        2410 non-null float64
<CLOSE>      2410 non-null float64
<TICKVOL>    2410 non-null int64
dtypes: float64(4), int64(1)
memory usage: 94.2 KB
    <OPEN>   <HIGH>    <LOW>  <CLOSE>  <TICKVOL>
0  108.802  108.818  108.783  108.795         75
1  108.795  108.822  108.778  108.799        244
2  108.795  108.817  108.785  108.794        122
3  108.802  108.805  108.778  108.784        328
4  108.784  108.793  108.781  108.786        529


In [43]:
def prepareData(df):
    #creating train and test sets
    start_date = '1999-01-04'
    start = 365 * 10 #2015
    # df.columns.drop(['Unnamed: 0', 'date'])

    dataset = df.values

#     dataset = np.array(sorted(dataset, key=lambda p: p[0], reverse=False))
    print(dataset)
    dataset = dataset.astype(np.float64)
    print(dataset.shape)

    train_size = int(len(dataset)*0.8)

    train = dataset[:train_size, :]
    valid = dataset[train_size:, :]
    #converting dataset into x_train and y_train
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(train)
    scaled_data_test = scaler.fit_transform(valid)

    x_train, y_train = [], []
    x_test, y_test = [], []

    lookback = 24*6 # one day
    for i in range(lookback,len(train)):
        x_train.append(scaled_data[i-lookback:i,:])
        y_train.append(scaled_data[i,1])
    for i in range(lookback,len(valid)):
        x_test.append(scaled_data_test[i-lookback:i,:])
        y_test.append(scaled_data_test[i,1])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_test, y_test = np.array(x_test), np.array(y_test)

#     x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],5))
#     x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],5))
    print(x_train.shape)
    print(x_test.shape)
    print(y_train.shape)
    return x_train, y_train, x_test, y_test, scaler

In [58]:
def buildModel(curr, x_train, y_train):
    i = 0
    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],5)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    X = x_train
    y = y_train
#     X = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
#     y = y_train

    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(X, y, epochs=1, batch_size=1, verbose=1)
    filename = curr + '_model.sav'
    pickle.dump(model, open(filename, 'wb'))

    i += 1
    

In [59]:
def loadModel(curr):
    i = 0

    # load the model from disk
    filename = curr + '_model.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    return loaded_model

In [60]:
def predict(x_test, model):
#     predict_arr = np.zeros((x_test.shape[0], len(models)))
    predict_arr = np.zeros((x_test.shape[0], 1))
    i = 0
#     X = x_test[:, :, i].reshape(x_test.shape[0], x_test.shape[1], 1)

    prediction = model.predict(x_test)
#     predict_arr = prediction.reshape(prediction.shape[0],)
    i += 1
#     for model in models:
#         X = x_test[:, :, i].reshape(x_test.shape[0], x_test.shape[1], 1)
        
#         prediction = model.predict(X)
#         predict_arr[:, i] = prediction.reshape(prediction.shape[0],)
#         i += 1
    return prediction

In [61]:
def getMSE(x_train, y_train, model, scaler):
    mse_arr = []
#     for model in models:
    X = x_train #[:, :, i].reshape(x_train.shape[0], x_train.shape[1], 1)
    p = model.predict(X)

    y_train_inv = scaler.inverse_transform(y_train)
    predict = scaler.inverse_transform(p)

    mse = mean_squared_error(y_train_inv, predict)

    return mse

In [62]:
df = pd.read_pickle(curr)
df.head(5)

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>
0,108.802,108.818,108.783,108.795,75
1,108.795,108.822,108.778,108.799,244
2,108.795,108.817,108.785,108.794,122
3,108.802,108.805,108.778,108.784,328
4,108.784,108.793,108.781,108.786,529


In [63]:
x_train, y_train, x_test, y_test, scaler = prepareData(df)

[[ 108.802  108.818  108.783  108.795   75.   ]
 [ 108.795  108.822  108.778  108.799  244.   ]
 [ 108.795  108.817  108.785  108.794  122.   ]
 ...
 [ 109.486  109.531  109.485  109.514  969.   ]
 [ 109.514  109.636  109.514  109.628 1075.   ]
 [ 109.628  109.628  109.608  109.608   88.   ]]
(2410, 5)
(1784, 144, 5)
(338, 144, 5)
(1784,)


In [64]:
buildModel(curr, x_train, y_train)

Epoch 1/1
1784/1784 [==============================] - 160s 90ms/step - loss: 0.0020


In [65]:
models = loadModel(curr)

In [66]:
predict_arr = predict(x_test, models)

In [70]:
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1)) ## scaler taking 5 data, currently y has 1 only
predict = scaler.inverse_transform(predict_arr)
curr_key = ['Canadian Dollar', 'Sterling', 'INR', 'US Dollar']
mse_arr = getMSE(x_train, y_train, models, scaler)
print(mse_arr)
horizon = 6*24*2 #2 days
for i in range(1):
    xs = np.arange(int(horizon))
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)

    ax.plot(xs, y_test_inv[-horizon:, i], label='Real ' + curr, lw=2, color='blue')
    ax.plot(xs, predict[-horizon:, i], label='Forecast ' + curr, lw=2, color='red')

    rmse = mse_arr[i]
    rmse_max = [float(p) + float(rmse)/2 for p in predict[:, i]]
    rmse_min = [float(p) - float(rmse)/2 for p in predict[:, i]]

    ax.fill_between(xs, rmse_max[-horizon:], rmse_min[-horizon:], color='red', alpha=0.2)
    plt.show()

ValueError: non-broadcastable output operand with shape (338,1) doesn't match the broadcast shape (338,5)

In [ ]:
# plt.plot(history.history['loss'], label='train_loss')
# plt.plot(history.history['val_loss'], label='test_loss')
# plt.legend()
# plt.show()
